In [2]:
using JLD2 
using Plots
using StatsBase

In [9]:
f = jldopen("../../Data/NASA_KeplerV2/datasets/KeplerLightCurveOrig.jld2", "r");
X_train = read(f, "X_train");
X_test = read(f, "X_test");
y_train = read(f, "y_train")
y_test = read(f, "y_test");

In [43]:
# isolate class 0 (contact EB)
contact_eb_tr_idxs = findall(x -> x .== 0, y_train);
contact_eb_te_idxs = findall(x -> x .==0, y_test);
X_train_contact_eb = X_train[contact_eb_tr_idxs, :]
X_test_contact_eb = X_test[contact_eb_te_idxs, :];
y_train_contact_eb = zeros(Int, size(X_train_contact_eb, 1));
y_test_contact_eb = zeros(Int, size(X_test_contact_eb, 1));

In [55]:
function detect_flat_regions(data)
    flat_regions = []
    for i in 2:length(data)
        if data[i] == data[i-1]
            push!(flat_regions, i)
        end
    end
    return flat_regions
end

detect_flat_regions (generic function with 1 method)

In [111]:
function make_overlapping_windows(ts::Vector{Float64}, window_size::Int, stride::Int)
    n = length(ts)
    windows = [ts[i:i+window_size-1] for i in 1:stride:n-window_size+1]
    return windows
end

make_overlapping_windows (generic function with 1 method)

In [84]:
windows = make_overlapping_windows(samp, 100, 90)

52-element Vector{Vector{Float64}}:
 [-0.00694375500418676, -0.00725043189343111, -0.0068968261992336, -0.00590279036643915, -0.00439117334115757, -0.00238330162894307, -0.000116945987635897, 0.00224155433407358, 0.00463202495360682, 0.00680999289433604  …  0.00923589345304188, 0.00907062672623626, 0.00778792477258305, 0.0058565846739328, 0.00356989280550335, 0.00118958243402256, -0.00113829417322775, -0.00333300829768313, -0.00515090573198218, -0.00651232844634098]
 [0.00923589345304188, 0.00907062672623626, 0.00778792477258305, 0.0058565846739328, 0.00356989280550335, 0.00118958243402256, -0.00113829417322775, -0.00333300829768313, -0.00515090573198218, -0.00651232844634098  …  -0.0070560914201484, -0.00655708067232863, -0.00545701137333643, -0.00381305535954801, -0.00177616552088489, 0.000488484229525854, 0.00278885169254717, 0.00498153543690383, 0.00665080148378661, 0.0074300313873108]
 [-0.0070560914201484, -0.00655708067232863, -0.00545701137333643, -0.00381305535954801, -0.00177

In [115]:
windows

52-element Vector{Vector{Float64}}:
 [-0.00694375500418676, -0.00725043189343111, -0.0068968261992336, -0.00590279036643915, -0.00439117334115757, -0.00238330162894307, -0.000116945987635897, 0.00224155433407358, 0.00463202495360682, 0.00680999289433604  …  0.00923589345304188, 0.00907062672623626, 0.00778792477258305, 0.0058565846739328, 0.00356989280550335, 0.00118958243402256, -0.00113829417322775, -0.00333300829768313, -0.00515090573198218, -0.00651232844634098]
 [0.00923589345304188, 0.00907062672623626, 0.00778792477258305, 0.0058565846739328, 0.00356989280550335, 0.00118958243402256, -0.00113829417322775, -0.00333300829768313, -0.00515090573198218, -0.00651232844634098  …  -0.0070560914201484, -0.00655708067232863, -0.00545701137333643, -0.00381305535954801, -0.00177616552088489, 0.000488484229525854, 0.00278885169254717, 0.00498153543690383, 0.00665080148378661, 0.0074300313873108]
 [-0.0070560914201484, -0.00655708067232863, -0.00545701137333643, -0.00381305535954801, -0.00177

In [42]:
# jldopen("contact_eb.jld2", "w") do f
#     f["X_train"] = X_train_contact_eb
#     f["X_test"] = X_test_contact_eb
#     f["y_train"] = y_train_contact_eb
#     f["y_test"] = y_test_contact_eb
# end;